<a href="https://colab.research.google.com/github/rishabhjambhulkar/Summarizer/blob/main/Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [3]:
import tensorflow as tf

# Check if GPU is available
if tf.test.gpu_device_name():
    print('GPU device found: {}'.format(tf.test.gpu_device_name()))
else:
    print("No GPU device found")

# Print GPU device specifications
!nvidia-smi


GPU device found: /device:GPU:0
Fri Jun 23 05:55:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    27W /  70W |    387MiB / 15360MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                        

In [4]:
!pip install matplotlib

import tensorflow as tf
import torch

from transformers import pipeline, set_seed

import matplotlib.pyplot as plt

import pandas as pd
from datasets import load_dataset, load_metric
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import pandas as pd
import numpy as np

import nltk
from nltk.tokenize import sent_tokenize

nltk.download("punkt")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## [The CNN/DailyMail Dataset](https://huggingface.co/datasets/cnn_dailymail)

##  An important aspect of the dataset is that the summaries are abstractive and not extractive, which means that they consist of new sentences instead of simple excerpts.

Extractive Summarization: the extractive approach selects the most important phrases and lines from the documents. It then combines all the important lines to create the summary. So, in this case, every line and word of the summary actually belongs to the original document which is summarized.

Abstractive Summarization: The abstractive approach uses new phrases and terms that are different from the original document, keeping the meaning the same, just like how humans do in summarization. So, it is much harder than the extractive approach.

In [5]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", version="3.0.0")

print(f"Features in cnn_dailymail : {dataset['train'].column_names}")

  0%|          | 0/3 [00:00<?, ?it/s]

Features in cnn_dailymail : ['article', 'highlights', 'id']


In [6]:
sample = dataset["train"][1]
print(f"""
Article (excerpt of 500 characters, total length: {len(sample["article"])}):
""")
print(sample["article"][:500])
print(f'\nSummary (length: {len(sample["highlights"])}):')
print(sample["highlights"])


Article (excerpt of 500 characters, total length: 4051):

Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most s

Summary (length: 281):
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .


-------------------

## Text Summarization Pipelines


In [7]:
sample_text = dataset["train"][1]["article"][:500]

# We'll collect the generated summaries of each model in a dictionary
summaries = {}

### Summarization Baseline
The function baseline_summary_three_sent(text) takes a text as input and returns a summary of the text consisting of the first three sentences. It uses the sent_tokenize function from the NLTK library to split the text into sentences. The sentences are then joined together using line breaks to create the summary.

In [8]:
def baseline_summary_three_sent(text):
    return "\n".join(sent_tokenize(text)[:3])

In [9]:
summaries['baseline'] = baseline_summary_three_sent(sample_text)

summaries['baseline']

'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.\nHere, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.\nMIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."'

# huggingface pipeline

The pipelines are a great and easy way to use models for inference.

Stating ”summarization”: will return a `SummarizationPipeline`

”text-generation”: will return a TextGenerationPipeline

-----------------------

# GPT-2

We can use GPT-2 it to generate summaries by simply appending “TL;DR” at the end of the input text.

The expression “TL;DR” (too long; didn’t read) is often used on platforms like
Reddit to indicate a short version of a long post. We will start our
summarization experiment by re-creating the procedure of the original paper
with the pipeline() function from Transformers

We create a text generation pipeline and load the GPT-2 model:

In [10]:
from transformers import pipeline, set_seed

set_seed(42)

pipe = pipeline('text-generation', model = 'gpt2-medium' )

gpt2_query = sample_text + "\nTL;DR:\n"

pipe_out = pipe(gpt2_query, max_length = 512, clean_up_tokenization_spaces = True)

# This passes the gpt2_query to the text generation pipeline (pipe) to generate text.
# The max_length parameter specifies the maximum length of the generated text in tokens.
# The clean_up_tokenization_spaces parameter ensures that the generated text has proper spacing.





Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [11]:
pipe_out

[{'generated_text': 'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most s\nTL;DR:\nThe 10,000 inmates charged with various nonviolent, nonviolent embody a culture that treats prisoners like enemies and rejects their needs. The 10,000 inmates charged with various nonviolent, nonviolent embody a culture that treats prisoners like enemies and rejects their needs. Inmates, who will spend time in isolation until trial, are housed in tiny cells below and outside of the winged wings, many of which are bare except for plastic cuffs, beds and windows th

In [12]:
pipe_out[0]["generated_text"][len(gpt2_query) :]

"The 10,000 inmates charged with various nonviolent, nonviolent embody a culture that treats prisoners like enemies and rejects their needs. The 10,000 inmates charged with various nonviolent, nonviolent embody a culture that treats prisoners like enemies and rejects their needs. Inmates, who will spend time in isolation until trial, are housed in tiny cells below and outside of the winged wings, many of which are bare except for plastic cuffs, beds and windows that close. Many of the inmates that stand in line will have a hard time doing much more than looking at prison walls. The 10,000 inmates charged with various nonviolent, nonviolent illustrate how the prison system is broken and inhumane. At once cruel and dehumanizing, the system is deeply frustrating, isolating, dangerous and, in many cases, ineffective. At once cruel and dehumanizing, the system is deeply frustrating, isolating, dangerous andAlpha Alpha Female: From solitary confinement, to forced prostitution, to rape\nBeta 

In [13]:
summaries['gpt2'] = "\n".join(sent_tokenize(pipe_out[0]["generated_text"][len(gpt2_query) :]))

# T5

T5 (Text-To-Text Transfer Transformer) is a transformer model that is trained in an end-to-end manner with text as input and modified text as output, in contrast to BERT-style models that can only output either a class label or a span of the input. This text-to-text formatting makes the T5 model fit for multiple NLP tasks like Summarization, Question-Answering, Machine Translation, and Classification problems.

How T5 is different from BERT?
Both T5 and BERT are trained with MLM (Masked Language Model) approach.

What is MLM?

The MLM is a fill-in-the-blank task, where the model masks part of the input text and tries to predict what that masked word should be.

Example:

“I like to eat peanut butter and <MASK> sandwiches,”
“I like to eat peanut butter and jelly sandwiches,”


The only difference is that T5 replaces multiple consecutive tokens with the single Mask Keyword, unlike, BERT which uses Mask token for each word. This illustration is shown below.


### T5 expects a prefix before the input text to understand the task given by the user. For example,

- “summarize:” for the summarization,
- “cola sentence:” for the classification,
- “translate English to Spanish:” for the machine translation, etc.,


--------------


But here in this case, I can directly load T5 for summarization with the pipeline() function, which also takes care of formatting the inputs in the text-to-text format so we don’t
need to prepend them with "summarize":


In [14]:
pipe = pipeline('summarization', model = 't5-small' )

pipe_out = pipe(sample_text)

Your max_length is set to 200, but your input_length is only 130. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=65)


In [15]:
pipe_out

[{'summary_text': 'inmate housed on the "forgotten floor" where many mentally ill inmates are housed . the ninth floor of the Miami-dade pretrial detention facility is dubbed "forget floor" inmates with the most solitary confinement are jailed in the u.s.'}]

In [16]:
summaries['t5'] = 'n'.join(sent_tokenize(pipe_out[0]['summary_text']))

# BART

BART is a denoising autoencoder for pretraining sequence-to-sequence models. It is trained by (1) corrupting text with an arbitrary noising function, and (2) learning a model to reconstruct the original text. It uses a standard Transformer-based neural machine translation architecture.

That means, It uses a standard seq2seq/NMT architecture with a bidirectional encoder (like BERT) and a left-to-right decoder (like GPT). This means the encoder's attention mask is fully visible, like BERT, and the decoder's attention mask is causal, like GPT2.


This means that a fine-tuned BART model can take a text sequence (for example, English) as input and produce a different text sequence at the output (for example, French).

This type of model is relevant for machine translation, question-answering , text summarization, or sequence classification (categorizing input text sentences or tokens).

Another task is sentence entailment which, given two or more sentences, evaluates whether the sentences are logical extensions or are logically related to a given statement.

In [17]:
pipe = pipeline("summarization", model="facebook/bart-large-cnn")
pipe_out = pipe(sample_text)


Your max_length is set to 142, but your input_length is only 110. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=55)


In [18]:
pipe_out

[{'summary_text': 'Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor" Here, inmates with the most s are housed in Miami before trial.'}]

In [19]:
summaries["bart"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

In [20]:
summaries["bart"]

'Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill.\nThe ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor" Here, inmates with the most s are housed in Miami before trial.'

# PEGASUS

The PEGASUS model’s pre-training task is very similar to summarization, i.e. important sentences are removed and masked from an input document and are later generated together as one output sequence from the remaining sentences, which is fairly similar to a summary. In PEGASUS, several whole sentences are removed from documents during pre-training, and the model is tasked with recovering them. The Input for such pre-training is a document with missing sentences, while the output consists of the missing sentences being concatenated together. The advantage of this self-supervision is that you can create as many examples as there are documents without any human intervention, which often becomes a bottleneck problem in purely supervised systems.

In [21]:
!pip install sentencepiece

pipe = pipeline('summarization', model="google/pegasus-cnn_dailymail"  )

pipe_out = pipe(sample_text)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Your max_length is set to 128, but your input_length is only 106. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=53)


In [22]:
pipe_out

[{'summary_text': 'Soledad O\'Brien takes users inside a jail where many inmates are mentally ill .<n>Inmates with the most mental illness are housed on the "forgotten floor"<n>The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor"'}]

In [23]:
summaries["pegasus"] = pipe_out[0]["summary_text"].replace(" .<n>", ".\n")

In [24]:
## Comparing Different Summaries

In [25]:
print("GROUND TRUTH")

print(dataset['train'][1]['highlights'])


for model_name in summaries:
    print(model_name.upper())
    print(summaries[model_name])


GROUND TRUTH
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .
BASELINE
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.
Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.
MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."
GPT2
The 10,000 inmates charged with various nonviolent, nonviolent embody a culture that treats prisoners like enemies and rejects their needs.
The 10,000 inmates charged with various nonviolent, nonviolent embody a culture 


# SacreBLEU

The bleu_metric object is an instance of the Metric class, and works like an
aggregator: you can add single instances with add() or whole batches via
add_batch(). Once you have added all the samples you need to evaluate, you
then call compute() and the metric is calculated. This returns a dictionary with
several values, such as the precision for each n-gram, the length penalty, as
well as the final BLEU score. Let’s look at the example from before:

In [26]:

from datasets import load_metric

bleu_metric = load_metric("sacrebleu")

<ipython-input-26-eacc06f6d0e4>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric("sacrebleu")


In [27]:
bleu_metric.add(prediction = [summaries["pegasus"]], reference = [dataset['train'][1]['highlights'] ])

results = bleu_metric.compute(smooth_method = 'floor', smooth_value = 0 )

results['precision'] = [np.round(p , 2) for p in results['precisions'] ]

pd.DataFrame.from_dict(results, orient = 'index', columns = ['Value'] )

,Value
score,15.352415
counts,"[22, 8, 6, 5]"
totals,"[53, 52, 51, 50]"
precisions,"[41.509433962264154, 15.384615384615385, 11.76..."
bp,0.927306
sys_len,53
ref_len,57
precision,"[41.51, 15.38, 11.76, 10.0]"


# ROUGE

# ROUGE vs BLEU

Bleu measures precision: how much the words (and/or n-grams) in the machine generated summaries appeared in the human reference summaries.

Rouge measures recall: how much the words (and/or n-grams) in the human reference summaries appeared in the machine generated summaries.

### Interpretation of Rouge Score

ROUGE-n recall=40% means that 40% of the n-grams in the reference summary are also present in the generated summary.

--------

The ROUGE score was specifically developed for applications like
summarization where high recall is more important than just precision.5

The approach is very similar to the BLEU score in that we look at different n-grams
and compare their occurrences in the generated text and the reference texts.


The difference is that with ROUGE we check how many n-grams in the
reference text also occur in the generated text. For BLEU we looked at how
many n-grams in the generated text appear in the reference

In [28]:
rouge_metric = load_metric('rouge')

## ROUGE-N

With ROUGE-N, the N represents the n-gram that we are using. For ROUGE-1 we would be measuring the match-rate of unigrams between our model output and reference.

ROUGE-2 and ROUGE-3 would use bigrams and trigrams respectively.


## ROUGE-L

ROUGE-L measures the longest common subsequence (LCS) between our model output and reference. All this means is that we count the longest sequence of tokens that is shared between both:


In the HF Datasets implementation, two variations of ROUGE are
calculated: one calculates the score per sentence and averages it for the
summaries (ROUGE-L), and the other calculates it directly over the whole
summary (ROUGE-Lsum).


In [29]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

reference = dataset['train'][1]['highlights']

records = []

for model_name in summaries:
    rouge_metric.add(prediction = summaries[model_name], reference = reference )
    score = rouge_metric.compute()
    rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )
    print('rouge_dict ', rouge_dict )
    records.append(rouge_dict)

pd.DataFrame.from_records(records, index = summaries.keys() )

rouge_dict  {'rouge1': 0.365079365079365, 'rouge2': 0.14516129032258066, 'rougeL': 0.20634920634920634, 'rougeLsum': 0.2857142857142857}
rouge_dict  {'rouge1': 0.10852713178294573, 'rouge2': 0.025974025974025976, 'rougeL': 0.07751937984496124, 'rougeLsum': 0.10335917312661498}
rouge_dict  {'rouge1': 0.45454545454545453, 'rouge2': 0.186046511627907, 'rougeL': 0.25, 'rougeLsum': 0.3409090909090909}
rouge_dict  {'rouge1': 0.45652173913043476, 'rouge2': 0.13333333333333333, 'rougeL': 0.1956521739130435, 'rougeLsum': 0.3695652173913043}
rouge_dict  {'rouge1': 0.41758241758241765, 'rouge2': 0.17977528089887637, 'rougeL': 0.2857142857142857, 'rougeLsum': 0.3516483516483516}


,rouge1,rouge2,rougeL,rougeLsum
baseline,0.365079,0.145161,0.206349,0.285714
gpt2,0.108527,0.025974,0.077519,0.103359
t5,0.454545,0.186047,0.250000,0.340909
bart,0.456522,0.133333,0.195652,0.369565
pegasus,0.417582,0.179775,0.285714,0.351648


# Evaluationg on the TEST set of the CNN/DailyMail Dataset

In [30]:
def calculate_metric_on_baseline_test_ds(dataset, metric, column_text = 'article', column_summary = 'highlights' ):
    """
    This function calculates a specified metric on a baseline test dataset for a Natural Language Processing (NLP) task.
    It assumes the task is a text summarization task, where the goal is to generate a summary (e.g., highlights) from a text (e.g., article).

    Parameters:
    dataset (pandas.DataFrame): The test dataset. It should contain a column for the text and a column for the true summary.
    metric (datasets.Metric): The metric to calculate. This should be a metric object from the Hugging Face datasets library.
    column_text (str, optional): The name of the column in the dataset that contains the text. Defaults to 'article'.
    column_summary (str, optional): The name of the column in the dataset that contains the true summary. Defaults to 'highlights'.

    Returns:
    score (float): The calculated score of the metric on the test dataset.
    """
    summaries = [baseline_summary_three_sent(text) for text in dataset[column_text] ]

    metric.add_batch(predictions = summaries, references = dataset[column_summary] )

    score = metric.compute()
    return score

In [31]:
test_sampled = dataset['train'].shuffle(seed = 42).select(range(1000))

score = calculate_metric_on_baseline_test_ds(test_sampled, rouge_metric )

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame.from_dict(rouge_dict, orient = 'index' , columns = ['baseline'] ).T

,rouge1,rouge2,rougeL,rougeLsum
baseline,0.253995,0.100642,0.165754,0.231571


## Strategy to calculate the ROUGE Metric on test dataset for the other Models

In [45]:
from tqdm import tqdm
import torch

device = torch.device("cuda:0")

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements.

    Generator function to yield successive batch-sized chunks from list_of_elements.

    Parameters:
    list_of_elements (list): List of elements to be divided into chunks.
    batch_size (int): The size of each chunk.

    Yields:
    list: Batch-sized chunk from list_of_elements.

    """
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    """
    Function to calculate a specified metric on a test dataset for a Natural Language Processing (NLP) task.
    It assumes the task is a text summarization task, where the goal is to generate a summary from a text.

    Parameters:
    dataset (pandas.DataFrame): The test dataset. It should contain a column for the text and a column for the true summary.
    metric (datasets.Metric): The metric to calculate. This should be a metric object from the Hugging Face datasets library.
    model (transformers.PreTrainedModel): The transformer model to use for text generation.
    tokenizer (transformers.PreTrainedTokenizer): The tokenizer corresponding to the model.
    batch_size (int, optional): The size of the batches to use for processing. Defaults to 16.
    device (str, optional): The device to run the model on. Defaults to the output of torch.cuda.is_available().
    column_text (str, optional): The name of the column in the dataset that contains the text. Defaults to 'article'.
    column_summary (str, optional): The name of the column in the dataset that contains the true summary. Defaults to 'highlights'.

    Returns:
    score (float): The calculated score of the metric on the test dataset.
    """
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        '''  num_beams is a parameter used in the generate() method of the transformer model.
        It controls the number of beams to use during beam search decoding.Beam search is a common
        technique used in text generation tasks, including text summarization. It explores multiple
        candidate sequences in parallel and keeps track of the most promising sequences, referred to
        as beams. The number of beams determines how many different sequences are considered during
        decoding.. '''

        # Finally, we decode the generated texts,
        # replace the <n> token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

In [33]:

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_ckpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

score = calculate_metric_on_test_ds(test_sampled, rouge_metric,
                                   model_pegasus, tokenizer, batch_size=8)

rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)

# At the end, we compute and return the ROUGE scores.
pd.DataFrame(rouge_dict, index=["pegasus"])

100%|██████████| 125/125 [23:40<00:00, 11.37s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.47519,0.278492,0.373879,0.428834
